In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib import patches
import seaborn as sns
sns.set_theme()
import pandas as pd
import mne
from scipy.fftpack import fft
from numpy.fft import rfft, irfft, rfftfreq
from sklearn.cross_decomposition import CCA
from sklearn.decomposition import PCA
# import tensorflow as tf
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
# from meegkit import dss, ress
# from meegkit import sns as msns
# from meegkit.utils import unfold, rms, fold, tscov, matmul3d, snr_spectrum
from brainda.paradigms import SSVEP
from brainda.algorithms.utils.model_selection import (
    set_random_seeds, 
    generate_loo_indices, match_loo_indices)
from brainda.algorithms.decomposition import (
    FBTRCA, FBTDCA, FBSCCA, FBECCA, FBDSP,
    generate_filterbank, generate_cca_references)
from collections import OrderedDict
import numpy as np
from scipy.signal import sosfiltfilt
from sklearn.pipeline import clone
from sklearn.metrics import balanced_accuracy_score

In [3]:
def load_data_temp_function(eeg, meta, classes, stim_duration=5, filter=True):
    # removes the first row and the last column of the meta.csv data
    trials = meta[1:,:2]
    times = []
    
    duration_samples = int(stim_duration*300)
    # this for-loop iterates through every row of the EEG data where TRG equals 16 and appends the time data at that row
    for index, row in eeg.loc[eeg[' TRG']==1.0].iterrows():
        if index > 0 and eeg.iloc[index-1][' TRG'] == 0 and (not eeg.iloc[index:index+duration_samples][' TRG'].isin([18.0]).any()):
            times.append(row['time'])
    times = np.array(times)
    

    # drop the columns time, TRG, X1, X2, X3, and A2.
    # Append the resulting table of length (stim_duration * 300) to the resulting eeg numpy array
    # for each place where TRG equals 16 after transposing
    eeg = np.array([eeg.loc[eeg['time']>t].drop(columns=['time',' TRG']).to_numpy()[:duration_samples].T for t in times])
    
    # if filter is set equal to true, the MNE filter, whose documentation can be found online at
    # https://mne.tools/stable/generated/mne.filter.filter_data.html
    # Performs a lower pass-band FIR filter on the EEG data
    if filter:
        eeg = mne.filter.filter_data(eeg, sfreq=300, l_freq=5, h_freq=49, verbose=0, method='fir')
    
    # classes is a numpy array of every unique row in meta.csv
    # so eeg_temp is a 2d list with a row for every row in classes
    eeg_temp = []
    for i in range(len(classes)):
        eeg_temp.append([])

    # populates the eeg_temp table with the elements of eeg grouped into rows by the corresponding 'class'
    for i,freq in enumerate(trials):
        for j,target in enumerate(classes):
            if (freq==target).all():
                eeg_temp[j].append(eeg[i])
    # switches the first and second dimensions of the eeg_temp array
    eeg = np.array(eeg_temp).transpose(1,0,2,3)
    return eeg

In [4]:
# list of sub-directories within the Aidan's DSI-24 data for each run
sub_dirs = ['run1/']
duration = 1.5
n_trials = 2
n_classes = 4
n_channels = 7

# load experimental dataset
# eeg_whole is a 4-dimensional numpy array where the first dimension is the number of trials times the number of runs
# each entry in the second class represents a different unique class (unique rows in the meta.csv data)
# third dimension represents the specific channel, which there are 19 of in this case because of the DSI-24
eeg_whole = np.zeros((n_trials*len(sub_dirs),n_classes,n_channels,int(duration*300)))

# target_tab stores the classes and their corresponding index in a dictionary
target_tab = {}
# i_class = 0
for i_dir,sub_dir in enumerate(sub_dirs):
    print(i_dir)
    # read the eeg and meta data from their csv files
    data_path = "../data/eeg_recordings/32-class_speller/DSI-7/Simon/" + sub_dir
    eeg = pd.read_csv(data_path + 'eeg.csv').astype(float)
    meta = np.loadtxt(data_path + 'meta.csv', delimiter=',', dtype=float)
    trials = meta[1:,:2]
    classes = np.unique(trials, axis=0)
    more_targets = {tuple(target):index for index,target in enumerate(classes)}
    # add current directory's classes with indexes to target_tab dictionary
    target_tab.update(more_targets)
    # call the load data function from block 2
    eeg = load_data_temp_function(eeg, meta, classes, stim_duration=duration,filter=False)
    # insert this run's eeg data into the eeg_whole numpy array
    eeg_whole[i_dir*n_trials:(i_dir+1)*n_trials,:,:,:] = eeg
    # i_class+=3
eeg = eeg_whole
# target by trial stores the classes, but repeated for each trial
target_by_trial = [list(target_tab.keys())] * n_trials*len(sub_dirs)
eeg.shape, np.array(target_by_trial).shape

0


((2, 32, 7, 450), (2, 32, 2))

In [ ]:
n_trials = eeg.shape[0]
# classes = [0,7,15,23,31]
classes = range(4)
n_classes = len(classes)
y = np.array([list(target_tab.values())] * n_trials).T.reshape(-1)
eeg_temp = eeg[:n_trials,classes,:,42:]
# X = eeg.swapaxes(0,1).reshape(-1,*eeg.shape[2:])
X = eeg_temp.swapaxes(0,1).reshape(-1,*eeg_temp.shape[2:])


freq_targets = np.array(target_by_trial)[0,:]
phase_targets = np.zeros_like(freq_targets)
n_harmonics = 5
n_bands = 3
srate = 300
duration = 1
Yf = generate_cca_references(
    freq_targets, srate, duration,
    phases=phase_targets,
    n_harmonics=n_harmonics)
wp = [[8*i, 90] for i in range(1, n_bands+1)]
ws = [[8*i-2, 95] for i in range(1, n_bands+1)]
filterbank = generate_filterbank(
    wp, ws, srate, order=4, rp=1)
filterweights = np.arange(1, len(filterbank)+1)**(-1.25) + 0.25
set_random_seeds(64)
l_val = 5
fbtdca_model = FBTDCA(filterbank, l_val, n_components=8, filterweights=filterweights)
events = []
# for j_class in range(32):
for j_class in classes:
    events.extend([str(target_by_trial[i_trial][j_class]) for i_trial in range(n_trials)])
events = np.array(events)
subjects = ['1'] * (n_classes*n_trials)
meta = pd.DataFrame(data=np.array([subjects,events]).T, columns=["subject", "event"])
set_random_seeds(42)

filterX, filterY = np.copy(X[..., :int(srate*duration) + l_val]), np.copy(y)
filterX = filterX - np.mean(filterX, axis=-1, keepdims=True)
trainX, trainY = filterX, filterY

model = fbtdca_model.fit(
    trainX, trainY,
    Yf=Yf
)

In [ ]:
pred_labels = model.predict(trainX)
score = balanced_accuracy_score(trainY, pred_labels)
print("Training Accuracy: " + str(score))

In [ ]:
import pickle
with open("../reports/trained_models/32-class_speller/DSI-24/Aidan/fbtdca_1s.pkl", 'wb') as filehandler:
    pickle.dump(model, filehandler)

In [ ]:
filterX.shape, X.shape, eeg_temp.shape